In [1]:
import ciftify
from ciftify.utils import run
import os
import datetime
import logging
import glob

In [2]:
#working_dir = '/home/edickie/Documents/ciftify_tests/'
working_dir = '/scratch/edickie/ciftify_intergration_tests/'
src_data_dir= os.path.join(working_dir,'src_data')
#work_from = '/home/edickie/Documents/ciftify_tests/run_2017-09-08/'
work_from = '/scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/'
#work_from = None

fixtures_dir = '/projects/edickie/code/ciftify/tests/integration/fixtures/'

if work_from:
    new_outputs = work_from
else:
    new_outputs= os.path.join(working_dir,'run_{}'.format(datetime.date.today()))
DEBUG = True
DRYRUN = True

## getting the data

freesurfer_webtgz = 'https://s3.amazonaws.com/openneuro/ds000030/ds000030_R1.0.4/compressed/ds000030_R1.0.4_derivatives_freesurfer_sub50004-50008.zip'
func_webtgz = 'https://s3.amazonaws.com/openneuro/ds000030/ds000030_R1.0.4/compressed/ds000030_R1.0.4_derivatives_sub50004-50008.zip'

subids = ['sub-50005','sub-50007']
#subids = ['sub-50005','sub-50006']

logger = logging.getLogger('ciftify')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

# Get settings, and add an extra handler for the subject log
fh = logging.FileHandler(os.path.join(working_dir, 'ciftify_tests.log'))
logger.addHandler(fh)

fs_subjects_dir = os.path.join(src_data_dir, 'ds000030_R1.0.4',
                               'derivatives','freesurfer')
hcp_data_dir = os.path.join(new_outputs, 'hcp')

if not os.path.exists(hcp_data_dir):
    run(['mkdir','-p',hcp_data_dir])

In [3]:
def download_file(web_address, local_filename):
    '''download file if it does not exist'''
    if not os.path.isfile(local_filename):
        run(['wget', web_address, '-O', local_filename])
    if not os.path.getsize(local_filename) > 0:
        os.remove(local_filename)

In [4]:
def get_recon_all_outputs(hcp_data_dir, subid):
    recon_all_out = folder_contents_list(os.path.join(hcp_data_dir, subid))
    recon_all_out = [x.replace(subid, 'subid') for x in recon_all_out]
    return(recon_all_out)

In [5]:
def folder_contents_list(path):
    '''returns a list of folder contents'''
    folder_contents = glob.glob(os.path.join(path, '**'), recursive = True)
    folder_contents = [x.replace('{}/'.format(path),'') for x in folder_contents ]
    folder_contents = folder_contents[1:] ## the first element is the path name
    return(folder_contents)

In [6]:
def seed_corr_default_out(func, seed):
    func = ciftify.meants.NibInput(func)
    seed = ciftify.meants.NibInput(seed)
    outputdir = os.path.dirname(func.path)
    outbase = '{}_{}'.format(func.base, seed.base)
    outputname = os.path.join(outputdir, outbase)
    if func.type == "nifti": outputname = '{}.nii.gz'.format(outputname)
    if func.type == "cifti": outputname = '{}.dscalar.nii'.format(outputname)
    return(outputname)

def run_vis_map(result_map, result_prefix, result_type):
    run(['cifti_vis_map', '{}-snaps'.format(result_type), '--hcp-data-dir', hcp_data_dir, 
     result_map, subid, result_prefix])

def run_seedcorr_peaktable(result_map):
    run(['ciftify_peaktable',  
         '--min-threshold', '-0.5', 
         '--max-threshold', '0.5', 
         '--no-cluster-dlabel', 
         result_map])

def run_seed_corr_fmri_test(func_cifti, hcp_data_dir, roi_dir):
    ''' runs one of the seed corrs and then peak table to get a csv to test for each dtseries'''
    subid = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(func_cifti)))))
    atlas_vol =  os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'wmparc.nii.gz')
    struct = 'RIGHT-PUTAMEN'
    putamen_vol_seed_mask = os.path.join(roi_dir, '{}_{}_vol.nii.gz'.format(subid, struct))
    if not os.path.exists(putamen_vol_seed_mask):
        run(['wb_command', '-volume-label-to-roi', atlas_vol, putamen_vol_seed_mask, '-name', struct])
    run(['ciftify_seed_corr', func_cifti, putamen_vol_seed_mask])
    run_seedcorr_peaktable(seed_corr_default_out(func_cifti, putamen_vol_seed_mask))

# Getting ABIDE and running recon-all

In [ ]:
abide_amazon_addy = 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1'

subid = 'NYU_0050954'
abide_freesurfer = os.path.join(src_data_dir, 'abide', 'freesurfer')

fs_subdir = os.path.join(abide_freesurfer, subid)
if not os.path.exists(fs_subdir):    
    run(['mkdir', '-p', fs_subdir])

for subdir in ['mri', 'surf','label','scripts']:
    localdir = os.path.join(fs_subdir, subdir)
    if not os.path.exists(localdir):
        run(['mkdir', '-p', localdir])
    
for filename in ['T1.mgz', 'aparc+aseg.mgz', 'aparc.a2009s+aseg.mgz', 'wmparc.mgz', 'brain.finalsurfs.mgz']:
    download_file(os.path.join(abide_amazon_addy, subid, 'mri', filename.replace('+','%20')),
                               os.path.join(fs_subdir, 'mri', filename))

for surface in ['pial', 'white', 'sphere.reg', 'sphere', 'curv', 'sulc', 'thickness']:
    for hemi in ['lh', 'rh']:
        download_file(os.path.join(abide_amazon_addy, subid, 'surf', '{}.{}'.format(hemi,surface)),
                      os.path.join(fs_subdir, 'surf', '{}.{}'.format(hemi,surface)))

for labelname in ['aparc', 'aparc.a2009s', 'BA']:
    for hemi in ['lh', 'rh']:
        download_file(os.path.join(abide_amazon_addy, subid, 'label', '{}.{}.annot'.format(hemi,labelname)),
                      os.path.join(fs_subdir, 'label', '{}.{}.annot'.format(hemi,labelname))) 

for script in ['recon-all.done', 'build-stamp.txt']:
    download_file(os.path.join(abide_amazon_addy, subid, 'scripts', script), 
                  os.path.join(fs_subdir, 'scripts', script))

In [ ]:
run(['ciftify_recon_all', '--hcp-data-dir', hcp_data_dir,
        '--fs-subjects-dir', abide_freesurfer, 
        'NYU_0050954'])

In [ ]:

run(['cifti_vis_recon_all', 'snaps', '--hcp-data-dir',hcp_data_dir, 'NYU_0050954'])
run(['cifti_vis_recon_all', 'index', '--hcp-data-dir', hcp_data_dir])

## Download ABIDE PCP data for ciftify_vol_result tests

In [ ]:
def download_vmhc(subid):
    amazon_addy = 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/cpac/filt_global/vmhc/{}_vmhc.nii.gz'.format(subid)
    sub_vmhc = os.path.join(src_vmhc, '{}_vmhc.nii.gz'.format(subid))
    download_file(amazon_addy, sub_vmhc)        

src_vmhc = os.path.join(src_data_dir, 'abide', 'vmhc')    
if not os.path.exists(src_vmhc): 
    run(['mkdir', src_vmhc])

subjects=['NYU_0050954','NYU_0050955']
for subid in subjects:
    download_vmhc(subid)

In [ ]:
subject = 'NYU_0050954'
run(['cifti_vis_map', 'nifti-snaps',
        '--hcp-data-dir', hcp_data_dir,
        '--qcdir', os.path.join(hcp_data_dir, 'abide_vmhc_vis'),
        '--resample-nifti', '--colour-palette', 'fidl', 
        os.path.join(src_vmhc, '{}_vmhc.nii.gz'.format(subject)), subject, '{}_vmhc'.format(subject)])

In [ ]:
subject = 'NYU_0050954'
run(['cifti_vis_map', 'nifti-snaps',
        '--hcp-data-dir', hcp_data_dir,
        '--qcdir', os.path.join(hcp_data_dir, 'abide_vmhc_vis'),
        '--resample-nifti',  
        os.path.join(src_vmhc, '{}_vmhc.nii.gz'.format(subject)), subject, '{}_vmhc_dcol'.format(subject)])

In [ ]:
subject = 'NYU_0050954'
run(['cifti_vis_map', 'nifti-snaps', 
        '--qcdir', os.path.join(hcp_data_dir, 'abide_vmhc_vis'),
        '--resample-nifti', '--colour-palette', 'fidl', 
        os.path.join(src_vmhc, '{}_vmhc.nii.gz'.format(subject)), 'HCP_S1200_GroupAvg', '{}_vmhc'.format(subject)])

In [ ]:
subject = 'NYU_0050955'
run(['cifti_vis_map', 'nifti-snaps',
        '--qcdir', os.path.join(hcp_data_dir, 'abide_vmhc_vis'),
        '--resample-nifti',
        os.path.join(src_vmhc, '{}_vmhc.nii.gz'.format(subject)), 'HCP_S1200_GroupAvg', '{}_vmhc'.format(subject)])

In [ ]:
run(['cifti_vis_map', 'index',
        '--hcp-data-dir', '/tmp',
        '--qcdir', os.path.join(hcp_data_dir, 'abide_vmhc_vis')])

# Testing ciftify_vol_result with HCP data..if data in src

In [ ]:
HCP_subid = '100307'
HCP_src_dir = os.path.join(src_data_dir, 'HCP') 
label_vol = os.path.join(HCP_src_dir, HCP_subid, 'MNINonLinear', 'aparc+aseg.nii.gz')
scalar_vol = os.path.join(HCP_src_dir, HCP_subid, 'MNINonLinear', 'T2w_restore.nii.gz')

run_HCP_tests = True if os.path.exists(HCP_src_dir) else False

if run_HCP_tests:
    HCP_out_dir = os.path.join(new_outputs, 'HCP')
    run(['mkdir', '-p', HCP_out_dir])
    run(['ciftify_vol_result', '--debug','--HCP-Pipelines', '--resample-nifti',
         '--hcp-data-dir', HCP_src_dir,
        '--integer-labels', HCP_subid, label_vol, 
         os.path.join(HCP_out_dir, '{}.aparc+aseg.32k_fs_LR.dscalar.nii'.format(HCP_subid))])

In [ ]:
if run_HCP_tests:
    run(['ciftify_vol_result','--HCP-Pipelines', '--HCP-MSMAll','--resample-nifti',
         '--hcp-data-dir', HCP_src_dir,
        '--integer-labels', HCP_subid, label_vol, 
         os.path.join(HCP_out_dir, '{}.aparc+aseg_MSMAll.32k_fs_LR.dscalar.nii'.format(HCP_subid))])

In [ ]:
if run_HCP_tests:
    run(['ciftify_vol_result','--HCP-Pipelines', '--resample-nifti',
         '--hcp-data-dir', HCP_src_dir, HCP_subid, 
         os.path.join(HCP_src_dir, HCP_subid, 'MNINonLinear', 'T2w_restore.nii.gz'),
         os.path.join(HCP_out_dir, '{}.T2w_restore.32k_fs_LR.dscalar.nii'.format(HCP_subid))])

In [ ]:
if run_HCP_tests:
    run(['ciftify_vol_result','--HCP-Pipelines', '--HCP-MSMAll',
         '--hcp-data-dir', HCP_src_dir, HCP_subid, 
         os.path.join(HCP_src_dir, HCP_subid, 'MNINonLinear', 'T2w_restore.2.nii.gz'),
         os.path.join(HCP_out_dir, '{}.T2w_restore_MSMAll.32k_fs_LR.dscalar.nii'.format(HCP_subid))])

# Download the main dataset

In [ ]:
subids = ['sub-50005','sub-50007']

if not os.path.exists(src_data_dir):
    run(['mkdir','-p',src_data_dir])

for subid in subids:
    sub_fs_path = os.path.join('ds000030_R1.0.4','derivatives','freesurfer', subid)
if not os.path.exists(os.path.join(src_data_dir, sub_fs_path)):
    if not os.path.exists(os.path.join(src_data_dir, 
                                       os.path.basename(freesurfer_webtgz))):        
        run(['wget', '-P', src_data_dir, freesurfer_webtgz])

    run(['unzip', 
         os.path.join(src_data_dir, 
                      os.path.basename(freesurfer_webtgz)), 
         os.path.join(sub_fs_path,'*'),
         '-d', src_data_dir])

for subid in subids:
    sub_file = os.path.join('ds000030_R1.0.4','derivatives',
                            'fmriprep', subid,'func',
                            '{}_task-rest_bold_space-T1w_preproc.nii.gz'.format(subid))
    if not os.path.exists(os.path.join(src_data_dir,sub_file)):
        if not os.path.exists(os.path.join(src_data_dir, 
                                           os.path.basename(func_webtgz))):
            run(['wget', '-P', src_data_dir, func_webtgz])

        run(['unzip', 
             os.path.join(src_data_dir, 
                          os.path.basename(func_webtgz)), 
             sub_file,
             '-d', src_data_dir])

for subid in subids:
    sub_file = os.path.join('ds000030_R1.0.4','derivatives',
                            'fmriprep', subid,'func',
                            '{}_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'.format(subid))
    if not os.path.exists(os.path.join(src_data_dir,sub_file)):
        if not os.path.exists(os.path.join(src_data_dir, 
                                           os.path.basename(func_webtgz))):
            run(['wget', '-P', src_data_dir, func_webtgz])

        run(['unzip', 
             os.path.join(src_data_dir, 
                          os.path.basename(func_webtgz)), 
             sub_file,
             '-d', src_data_dir])

In [ ]:
run(['ciftify_recon_all', '--resample-to-T1w32k', '--hcp-data-dir', hcp_data_dir,
    '--fs-subjects-dir', fs_subjects_dir, 
    subids[0]])

In [ ]:
run(['ciftify_recon_all', '--hcp-data-dir', hcp_data_dir,
    '--fs-subjects-dir', fs_subjects_dir, 
    subids[1]])

In [ ]:
for subid in subids:
    run(['cifti_vis_recon_all', 'snaps', '--hcp-data-dir',hcp_data_dir, subid])
run(['cifti_vis_recon_all', 'index', '--hcp-data-dir', hcp_data_dir])

In [ ]:
# for subid in ['sub-50004','sub-50006', 'sub-50008']:
#     run(['ciftify_recon_all', '--hcp-data-dir', hcp_data_dir,
#         '--fs-subjects-dir', fs_subjects_dir, 
#         subid])

In [ ]:
# for subid in ['sub-50004','sub-50006', 'sub-50008']:
#     run(['cifti_vis_recon_all', 'snaps', '--hcp-data-dir',hcp_data_dir, subid])
# run(['cifti_vis_recon_all', 'index', '--hcp-data-dir', hcp_data_dir])

# Running ciftify_subject_fmri 

In [7]:
subid=subids[0]
native_func = os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-native_preproc.nii.gz'.format(subid))
mat_file = os.path.join(src_data_dir, 'ds000030_R1.0.4','derivatives',
                            'fmriprep', subid,'func',
                            '{}_task-rest_bold_T1_to_EPI.mat'.format(subid))

with open(mat_file, "w") as text_file:
    text_file.write('''1.03  -0.015  0.0025  -15.0
0.014  1.01  -0.005  -11.9
-0.007 0.01  0.99  2
0  0  0  1
''')

t1_func = os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-T1w_preproc.nii.gz'.format(subid))
if not os.path.exists(native_func):
    run(['flirt', '-in', t1_func, '-ref', t1_func, 
         '-out', native_func, '-init', mat_file, '-applyxfm'])

run(['ciftify_subject_fmri', '--SmoothingFWHM', '12', 
    '--hcp-data-dir', hcp_data_dir, 
    native_func, subid, 'rest_test1'])

Running: ciftify_subject_fmri --SmoothingFWHM 12 --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp /scratch/edickie/ciftify_intergration_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-native_preproc.nii.gz sub-50005 rest_test1
cmd: ciftify_subject_fmri --SmoothingFWHM 12 --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp /scratch/edickie/ciftify_intergration_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-native_preproc.nii.gz sub-50005 rest_test1 
 Failed with returncode 1
Subject output already exits.
 To force rerun, delete or rename the logfile:
	/scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/ciftify_subject_fmri.log



1

In [ ]:
subid=subids[0]
run(['cifti_vis_fmri', 'snaps', '--SmoothingFWHM', '12', '--hcp-data-dir',hcp_data_dir, 'rest_test1', subid])

In [ ]:
subid=subids[0]
native_func_mean = os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-native_mean.nii.gz'.format(subid))

run(['fslmaths', native_func, '-Tmean', native_func_mean])

run(['ciftify_subject_fmri', '--hcp-data-dir', hcp_data_dir, 
    '--FLIRT-template', native_func_mean, native_func, subid, 'rest_test2'])


In [ ]:
subid=subids[0]
run(['cifti_vis_fmri', 'snaps', '--smooth-conn', '12', '--hcp-data-dir',hcp_data_dir, 'rest_test2', subid])

In [ ]:
subid = subids[1]
run(['ciftify_subject_fmri', '--SmoothingFWHM', '8', '--FLIRT-dof', '30',
    '--hcp-data-dir', hcp_data_dir, 
     '--OutputSurfDiagnostics',
    os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-T1w_preproc.nii.gz'.format(subid)),
    subid, 'rest_test_dof30'])

In [ ]:
subid = subids[1]
run(['cifti_vis_fmri', 'snaps', '--hcp-data-dir',hcp_data_dir, 'rest_test_dof30', subid])

In [ ]:
subid = subids[1]
run(['ciftify_subject_fmri', '--already-in-MNI', '--SmoothingFWHM', '8', 
    '--hcp-data-dir', hcp_data_dir, 
    os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'.format(subid)),
    subid, 'rest_bad_transform'])

In [ ]:
subid = subids[1]
run(['cifti_vis_fmri', 'snaps', '--hcp-data-dir',hcp_data_dir, 'rest_bad_transform', subid])

In [ ]:
run(['cifti_vis_fmri', 'index', '--hcp-data-dir', hcp_data_dir])

In [ ]:
from glob import glob
dtseries_files = glob(os.path.join(hcp_data_dir,'*',
                                       'MNINonLinear', 'Results',
                                       '*', '*Atlas_s*.dtseries.nii'))
run(['cifti_vis_RSN', 'cifti-snaps',  
         '--hcp-data-dir',hcp_data_dir, 
         '--colour-palette', 'PSYCH-NO-NONE',
         dtseries_files[0], 
         os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(dtseries_files[0])))))])
    
for dtseries in dtseries_files[1:]:
    run(['cifti_vis_RSN', 'cifti-snaps',  
         '--hcp-data-dir',hcp_data_dir, dtseries, 
         os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(dtseries)))))])
run(['cifti_vis_RSN', 'index', 
     '--hcp-data-dir', hcp_data_dir])

In [ ]:
run(['cifti_vis_RSN', 'index', '--hcp-data-dir', hcp_data_dir])

In [ ]:
roi_dir = os.path.join(new_outputs, 'rois')
if not os.path.exists(roi_dir):
    run(['mkdir', roi_dir])
for dtseries in dtseries_files:
    run_seed_corr_fmri_test(dtseries, hcp_data_dir, roi_dir)

In [ ]:
run(['cifti_vis_RSN', 'nifti-snaps', '--debug', 
         '--hcp-data-dir',hcp_data_dir, 
         '--qcdir', os.path.join(hcp_data_dir, 'RSN_from_nii'),
         '--colour-palette', 'PSYCH-NO-NONE',
     
        os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subids[0],'func',
                 '{}_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'.format(subids[0])),
        subids[0]])

run(['cifti_vis_RSN', 'nifti-snaps',  
         '--hcp-data-dir',hcp_data_dir, 
         '--qcdir', os.path.join(hcp_data_dir, 'RSN_from_nii'),
        os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subids[1],'func',
                 '{}_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'.format(subids[1])),
        subids[1]])
    
run(['cifti_vis_RSN', 'index', 
     '--qcdir', os.path.join(hcp_data_dir, 'RSN_from_nii'),
     '--hcp-data-dir', hcp_data_dir])

In [ ]:
groupmask_cmd = ['ciftify_groupmask', os.path.join(hcp_data_dir, 'groupmask.dscalar.nii')]
groupmask_cmd.extend(dtseries_files)

run(groupmask_cmd)     

# Running PINT and related functions

In [ ]:
subid = subids[0]
run(['mkdir', '-p', os.path.join(new_outputs, 'PINT', subid)])

run(['ciftify_PINT_vertices', '--pcorr', 
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test1', 'rest_test1_Atlas_s12.dtseries.nii'),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(ciftify.config.find_ciftify_global(), 'PINT', 'Yeo7_2011_80verts.csv'),
     os.path.join(new_outputs, 'PINT', subid, subid)])


In [ ]:
subid = subids[0]
run(['cifti_vis_PINT', 'snaps', '--hcp-data-dir',  hcp_data_dir,
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test1', 'rest_test1_Atlas_s12.dtseries.nii'),
     subid,
     os.path.join(new_outputs, 'PINT', subid, '{}_summary.csv'.format(subid))])

In [ ]:
subid = subids[1]
run(['mkdir', '-p', os.path.join(new_outputs, 'PINT', subid)])

run(['ciftify_PINT_vertices',
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test_dof30', 'rest_test_dof30_Atlas_s8.dtseries.nii'),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(ciftify.config.find_ciftify_global(), 'PINT', 'Yeo7_2011_80verts.csv'),
     os.path.join(new_outputs, 'PINT', subid, subid)])

In [ ]:
run(['cifti_vis_PINT', 'snaps', '--hcp-data-dir',  hcp_data_dir,
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test_dof30', 'rest_test_dof30_Atlas_s8.dtseries.nii'),
     subid,
     os.path.join(new_outputs, 'PINT', subid, '{}_summary.csv'.format(subid))])

In [ ]:
subid = subids[1]
run(['mkdir', '-p', os.path.join(new_outputs, 'PINT', subid)])

run(['ciftify_PINT_vertices', '--outputall',
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test_dof30', 'rest_test_dof30_Atlas_s8.dtseries.nii'),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(ciftify.config.find_ciftify_global(), 'PINT', 'Yeo7_2011_80verts.csv'),
     os.path.join(new_outputs, 'PINT', subid, '{}_all'.format(subid))])

In [ ]:
run(['cifti_vis_PINT', 'snaps', '--hcp-data-dir',  hcp_data_dir,
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test_dof30', 'rest_test_dof30_Atlas_s8.dtseries.nii'),
     subid,
     os.path.join(new_outputs, 'PINT', subid, '{}_all_summary.csv'.format(subid))])

In [ ]:
run(['cifti_vis_PINT', 'index', '--hcp-data-dir',  hcp_data_dir])

In [ ]:
summary_files = glob(os.path.join(new_outputs, 'PINT', '*','*_summary.csv'))
concat_cmd = ['ciftify_postPINT1_concat', '--debug',
           os.path.join(new_outputs, 'PINT', 'concatenated.csv')]
concat_cmd.extend(summary_files)
run(concat_cmd)

In [ ]:
run(['ciftify_postPINT2_sub2sub', os.path.join(new_outputs, 'PINT', 'concatenated.csv'),
     os.path.join(new_outputs, 'PINT', 'ivertex_distances.csv')])

In [ ]:
run(['ciftify_postPINT2_sub2sub', '--roiidx', '14',
     os.path.join(new_outputs, 'PINT', 'concatenated.csv'),
     os.path.join(new_outputs, 'PINT', 'ivertex_distances_roiidx14.csv')])

# Running ciftify_surface_rois

In [ ]:
vertices1_csv = os.path.join(src_data_dir, 'vertices1.csv')

with open(vertices1_csv, "w") as text_file:
    text_file.write('''hemi,vertex
L,11801
L,26245
L,26235
L,26257
L,13356
L,289
L,13336
L,13337
L,26269
L,13323
L,26204
L,26214
L,13326
L,13085
L,13310
L,13281
L,13394
L,13395
L,26263
L,26265
L,13343
L,77
L,13273
L,13342
L,26271
L,11804
L,13322
L,13369
L,13353
L,26268
L,26201
L,26269
L,68
L,13391
''')

subid = subids[0]
run(['mkdir', '-p', os.path.join(new_outputs, 'rois')])
run(['ciftify_surface_rois', vertices1_csv, '10', '--gaussian',
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)), 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii')])

In [ ]:
subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii'),
     subid, 
     'gaussian_roi'])

In [27]:
run(['ciftify_peaktable', '--debug', '--max-threshold', '0.05',      
    '--left-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
    '--right-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)),
    os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii')])

Running: ciftify_peaktable --debug --max-threshold 0.05 --left-surface /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii --right-surface /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf.gii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/gaussian_roi.dscalar.nii
[ciftify_peaktable] INFO: 

            .|';   ||          .|';
       ''   ||     ||     ''   ||
.|'',  ||  '||'  ''||''   ||  '||'  '||  ||`
||     ||   ||     ||     ||   ||    `|..||
`|..' .||. .||.    `|..' .||. .||.       ||
                                      ,  |'
                                       ''   
-------------------------------------------------------------
2017-10-16 12:12:36.040510 : Starting ciftify_peaktable
----------------------------------------

0

In [ ]:
vertices2_csv = os.path.join(src_data_dir, 'vertices1.csv')

with open(vertices2_csv, "w") as text_file:
    text_file.write('''hemi,vertex
R,2379
R,2423
R,2423
R,2629
R,29290
R,29290
R,1794
R,29199
R,1788
R,2380
R,2288
R,29320
R,29274
R,29272
R,29331
R,29356
R,2510
R,29345
R,2506
R,2506
R,1790
R,29305
R,2630
R,2551
R,2334
R,2334
R,29306
R,2551
R,2422
R,29256
R,2468
R,29332
R,2425
R,29356
''')
    
run(['mkdir', '-p', os.path.join(new_outputs, 'rois')])
run(['ciftify_surface_rois', vertices1_csv, '6', '--probmap',
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)), 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii')])

In [ ]:
subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii'),
     subid, 
     'probmap_roi'])

In [ ]:
run(['ciftify_peaktable', '--max-threshold', '0.05',      
    '--left-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
    '--right-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)), 
   os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii')])

In [ ]:
for hemi in ['L','R']:
    run(['wb_command', '-surface-vertex-areas',
         os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.{}.midthickness.32k_fs_LR.surf.gii'.format(subid, hemi)),
         os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.{}.midthickness_va.32k_fs_LR.shape.gii'.format(subid, hemi))])

run(['ciftify_peaktable', 
    '--outputbase', os.path.join(new_outputs, 'rois', 'probmap_roi_withva'),
    '--max-threshold', '0.05',      
    '--left-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
    '--right-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)), 
    '--left-surf-area', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness_va.32k_fs_LR.shape.gii'.format(subid)),
    '--right-surf-area', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness_va.32k_fs_LR.shape.gii'.format(subid)), 
   os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii')])

In [ ]:
run(['ciftify_surface_rois',
      os.path.join(ciftify.config.find_ciftify_global(), 'PINT', 'Yeo7_2011_80verts.csv'),
      '6', '--vertex-col', 'tvertex', '--labels-col', 'NETWORK', '--overlap-logic', 'EXCLUDE',
      os.path.join(ciftify.config.find_HCP_S1200_GroupAvg(),
                'S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii'),
      os.path.join(ciftify.config.find_HCP_S1200_GroupAvg(),
                'S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii'),
      os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

In [ ]:
subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii'),
     subid, 
     'tvertex'])

# Running ciftify_seed_corr 

In [9]:
subid = subids[0]
smoothing = 12
func_vol = os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'Results', 'rest_test1', 'rest_test1.nii.gz')
func_cifti_sm0 = os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'Results', 'rest_test1', 'rest_test1_Atlas_s0.dtseries.nii')
func_cifti_smoothed = os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'Results', 'rest_test1', 'rest_test1_Atlas_s{}.dtseries.nii'.format(smoothing))
atlas_vol =  os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'wmparc.nii.gz')
seed_corr_dir = os.path.join(new_outputs, 'ciftify_seed_corr')
if not os.path.exists(seed_corr_dir):
    run(['mkdir', '-p', seed_corr_dir])
cifti_mask = os.path.join(seed_corr_dir, '{}_func_mask.dscalar.nii'.format(subid))
run(['wb_command', '-cifti-math', "'(x > 0)'", cifti_mask,
    '-var', 'x', func_cifti_sm0, '-select', '1', '1'])

Running: wb_command -cifti-math '(x > 0)' /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii -var x /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii -select 1 1
parsed '(x > 0)' as '(x>0)'



0

In [ ]:
### All results
# ciftify_peaktable - with va input
# cifti_vis_map
run(['extract_nuisance_regressors', 
     os.path.join(hcp_data_dir, subid, 'MNINonLinear'), 
     func_vol])


In [10]:
hipp_struct = 'LEFT-HIPPOCAMPUS'
hipp_vol_seed_mask = os.path.join(seed_corr_dir, '{}_{}_vol.nii.gz'.format(subid, hipp_struct))
run(['wb_command', '-volume-label-to-roi', atlas_vol, hipp_vol_seed_mask, '-name', hipp_struct])

Running: wb_command -volume-label-to-roi /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/wmparc.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_LEFT-HIPPOCAMPUS_vol.nii.gz -name LEFT-HIPPOCAMPUS


0

In [ ]:
run(['ciftify_seed_corr', '--debug', '--fisher-z', func_vol, hipp_vol_seed_mask])

In [91]:
class MeantsTest(object):
    def __init__(self, func, seed, subid, sc_type, hcp_data_dir, outputdir = None, extra_args = None, mask = None):
        self.func = ciftify.meants.NibInput(func)
        self.seed = ciftify.meants.NibInput(seed)
        self.mask = mask
        self.outputdir = outputdir
        self.prefix = self.get_output_prefix(sc_type)
        self.result_map, self.result_csv = self.get_result_paths()
        self.seed_corr_pass = self.test_ciftify_seed_corr(extra_args)
        self.meants_pass = self.test_ciftify_meants(extra_args)
        self.vis_map_ran = self.run_vis_map(hcp_data_dir, subid)
        self.peaktable_pass = self.run_seedcorr_peaktable()
    
    def get_output_prefix(self, sc_type):
        ''' puts the other info together into an output prefix'''
        masked = "umasked" if self.mask == None else "masked" 
        prefix = "{}_{}_{}_{}".format(self.func.base,
                                     self.seed.base,
                                     sc_type,
                                     masked)
        return(prefix)
    
    def get_result_paths(self):
        ''' set outputdir to location of func if not speficified'''
        if self.outputdir:
            expected_path = self.outputdir
            outbase = self.prefix
        else:
            expected_path = os.path.dirname(self.func.path)
            outbase = '{}_{}'.format(self.func.base, self.seed.base)
        output_path = os.path.join(expected_path, outbase)
        if self.func.type == "cifti":
           result_map = "{}.dscalar.nii".format(output_path)
        elif self.func.type == "nifti":
           result_map = "{}.nii.gz".format(output_path)
        else:
          logger.error("func should be nifti or cifti")
        result_csv = "{}.csv".format(output_path)
        return(result_map, result_csv)
    
    def test_ciftify_seed_corr(self, extra_args):
        '''run ciftify_seed_corr'''
        sc_cmd = ['ciftify_seed_corr', self.func.path, self.seed.path]
        if extra_args:
            sc_cmd.insert(1, extra_args)
        if self.outputdir:
            sc_cmd.insert(1, '--outputname')
            sc_cmd.insert(2, self.result_map)
        if self.mask:
            sc_cmd.insert(1, '--mask')
            sc_cmd.insert(2, self.mask)
        run(sc_cmd)
        return(os.path.exists(self.result_map))
    
    def test_ciftify_meants(self, extra_args):
        ''' runs ciftify_meants'''
        meants_cmd = ['ciftify_meants', self.func.path, self.seed.path]
        if extra_args:
            meants_cmd.insert(1, extra_args)
        if self.outputdir:
            meants_cmd.insert(1, '--outputcsv')
            meants_cmd.insert(2, self.result_csv)
        if self.mask:
            meants_cmd.insert(1, '--mask')
            meants_cmd.insert(2, self.mask)
        run(meants_cmd)
        return(os.path.exists(self.result_csv))
        
    def run_vis_map(self, hcp_data_dir, subid):
        ''' runs vis map to make visual of this process'''
        run(['cifti_vis_map', '{}-snaps'.format(self.func.type), 
             '--hcp-data-dir', hcp_data_dir, 
             self.result_map, 
             subid, 
             self.prefix])
        return(True)

    def run_seedcorr_peaktable(self):
        ''' runs peaktable with correlation style threshold'''
        run(['ciftify_peaktable',  
             '--min-threshold', '-0.5', 
             '--max-threshold', '0.5', 
             '--no-cluster-dlabel', 
             self.result_map])
        return(True)
    
    

In [15]:
run(['ciftify_meants', '--debug', func_vol, hipp_vol_seed_mask])

Running: ciftify_meants --debug /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_LEFT-HIPPOCAMPUS_vol.nii.gz


            .|';   ||          .|';
       ''   ||     ||     ''   ||
.|'',  ||  '||'  ''||''   ||  '||'  '||  ||`
||     ||   ||     ||     ||   ||    `|..||
`|..' .||. .||.    `|..' .||. .||.       ||
                                      ,  |'
                                       ''   
-------------------------------------------------------------
2017-10-13 17:32:06.432133 : Starting ciftify_meants
-------------------------------------------------------------

Arguments:
    !!python/object/new:docopt.Dict
    dictitems:
      --debug: true
      --help: false
      --hemi: null
      --mask: null
      --outputcsv: null
      --outputlabels: null
      --roi-label: null
      -

0

In [9]:
run(['cifti_vis_map', 'nifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     seed_corr_default_out(func_vol, hipp_vol_seed_mask),
     subid, 
     '{}_{}_niftitoniftiZ_unmasked'.format(subid, hipp_struct)])

Running: cifti_vis_map nifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_sub-50005_LEFT-HIPPOCAMPUS_vol.nii.gz sub-50005 sub-50005_LEFT-HIPPOCAMPUS_niftitoniftiZ_unmasked


0

In [13]:
run(['ciftify_meants',
     '--outputcsv', os.path.join(seed_corr_dir, '{}_{}_niftitoniftiZ_masked.csv'.format(subid, hipp_struct)),
     '--mask', os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'brainmask_fs.nii.gz'),
     func_vol, hipp_vol_seed_mask])

Running: ciftify_meants --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_LEFT-HIPPOCAMPUS_niftitoniftiZ_masked.csv --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/brainmask_fs.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_LEFT-HIPPOCAMPUS_vol.nii.gz


0

In [13]:
run(['ciftify_meants',
     '--outputcsv', os.path.join(seed_corr_dir, '{}_{}_niftitoniftiZ_masked.csv'.format(subid, hipp_struct)),
     '--mask', os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'brainmask_fs.nii.gz'),
     func_vol, hipp_vol_seed_mask])

Running: ciftify_meants --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_LEFT-HIPPOCAMPUS_niftitoniftiZ_masked.csv --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/brainmask_fs.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_LEFT-HIPPOCAMPUS_vol.nii.gz


0

In [11]:
run(['ciftify_seed_corr', '--fisher-z',
     '--outputname', os.path.join(seed_corr_dir, '{}_{}_niftitoniftiZ_masked.nii.gz'.format(subid, hipp_struct)),
     '--mask', os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'brainmask_fs.nii.gz'),
     func_vol, hipp_vol_seed_mask])

run(['cifti_vis_map', 'nifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(seed_corr_dir, '{}_{}_niftitoniftiZ_masked.nii.gz'.format(subid, hipp_struct)),
     subid, 
     '{}_{}_niftitonifti_masked'.format(subid, hipp_struct)])

Running: ciftify_seed_corr --fisher-z --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_LEFT-HIPPOCAMPUS_niftitoniftiZ_masked.nii.gz --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/brainmask_fs.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_LEFT-HIPPOCAMPUS_vol.nii.gz
Running: cifti_vis_map nifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_LEFT-HIPPOCAMPUS_niftitoniftiZ_masked.nii.gz sub-50005 sub-50005_LEFT-HIPPOCAMPUS_niftitonifti_masked


0

In [14]:
run(['ciftify_seed_corr', '--fisher-z', func_cifti_smoothed, hipp_vol_seed_mask])

run(['ciftify_meants', func_cifti_smoothed, hipp_vol_seed_mask])

run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     seed_corr_default_out(func_cifti_smoothed, hipp_vol_seed_mask),
     subid, 
     '{}_{}_niftitociftiZ_unmasked'.format(subid, hipp_struct)])

run_seedcorr_peaktable(seed_corr_default_out(func_cifti_smoothed, hipp_vol_seed_mask))

Running: ciftify_seed_corr --fisher-z /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_LEFT-HIPPOCAMPUS_vol.nii.gz
Running: ciftify_meants /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_LEFT-HIPPOCAMPUS_vol.nii.gz
Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_LEFT-HIPPOCAMPUS_vol.dscalar.nii sub-50005 sub-50005_LEFT-HIPPOCAMPUS_nif

In [19]:
run(['ciftify_seed_corr', '--fisher-z',
     '--outputname', os.path.join(seed_corr_dir, '{}_{}_niftitociftiZ_masked.dscalar.nii'.format(subid, hipp_struct)),
     '--mask', cifti_mask,
     func_cifti_smoothed, hipp_vol_seed_mask])

run(['ciftify_meants',
     '--outputcsv', os.path.join(seed_corr_dir, '{}_{}_niftitociftiZ_masked.csv'.format(subid, hipp_struct)),
     '--mask', cifti_mask,
     func_cifti_smoothed, hipp_vol_seed_mask])

subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', hcp_data_dir, 
     os.path.join(seed_corr_dir, '{}_{}_niftitociftiZ_masked.dscalar.nii'.format(subid, hipp_struct)),
     subid, 
     '{}_{}_niftitociftiZ_masked'.format(subid, hipp_struct)])
run_seedcorr_peaktable(os.path.join(seed_corr_dir, '{}_{}_niftitociftiZ_masked.dscalar.nii'.format(subid, hipp_struct)))


Running: ciftify_seed_corr --fisher-z --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_LEFT-HIPPOCAMPUS_niftitociftiZ_masked.dscalar.nii --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_LEFT-HIPPOCAMPUS_vol.nii.gz
Running: ciftify_meants --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_LEFT-HIPPOCAMPUS_niftitociftiZ_masked.csv --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-

In [20]:
struct = 'RIGHT-PUTAMEN'
putamen_vol_seed_mask = os.path.join(seed_corr_dir, '{}_{}_vol.nii.gz'.format(subid, struct))
run(['wb_command', '-volume-label-to-roi', atlas_vol, putamen_vol_seed_mask, '-name', struct])

Running: wb_command -volume-label-to-roi /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/wmparc.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz -name RIGHT-PUTAMEN


0

In [21]:
run(['ciftify_seed_corr', func_vol, putamen_vol_seed_mask])
run(['ciftify_meants', func_vol, putamen_vol_seed_mask])
run(['cifti_vis_map', 'nifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     seed_corr_default_out(func_vol, putamen_vol_seed_mask),
     subid, 
     '{}_{}_niftitonifti_unmasked'.format(subid, struct)])

Running: ciftify_seed_corr /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz
Running: ciftify_meants /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz
Running: cifti_vis_map nifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_sub-50005_RIGHT-PUTAMEN_vol.nii.gz sub-50005 sub-50005_RIGHT-PUTAMEN_niftitonifti_unmasked


0

In [22]:
run(['ciftify_seed_corr',
     '--outputname', os.path.join(seed_corr_dir, '{}_{}_niftitonifti_masked.nii.gz'.format(subid, struct)),
     '--mask', os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'brainmask_fs.nii.gz'),
     func_vol, putamen_vol_seed_mask])

run(['ciftify_meants',
     '--outputcsv', os.path.join(seed_corr_dir, '{}_{}_niftitonifti_masked.csv'.format(subid, struct)),
     '--mask', os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'brainmask_fs.nii.gz'),
     func_vol, putamen_vol_seed_mask])

run(['cifti_vis_map', 'nifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(seed_corr_dir, '{}_{}_niftitonifti_masked.nii.gz'.format(subid, struct)),
     subid, 
     '{}_{}_niftitonifti_masked'.format(subid, struct)])

Running: ciftify_seed_corr --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_niftitonifti_masked.nii.gz --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/brainmask_fs.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz
Running: ciftify_meants --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_niftitonifti_masked.csv --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/brainmask_fs.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinea

0

In [23]:
run(['ciftify_seed_corr', func_cifti_smoothed, putamen_vol_seed_mask])

run(['ciftify_meants', func_cifti_smoothed, putamen_vol_seed_mask])

run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     seed_corr_default_out(func_cifti_smoothed, putamen_vol_seed_mask),
     subid, 
     '{}_{}_niftitocifti_unmasked'.format(subid, struct)])

run_seedcorr_peaktable(seed_corr_default_out(func_cifti_smoothed, putamen_vol_seed_mask))

Running: ciftify_seed_corr /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz
Running: ciftify_meants /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz
Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_RIGHT-PUTAMEN_vol.dscalar.nii sub-50005 sub-50005_RIGHT-PUTAMEN_niftitocifti_unmasked
Runn

In [24]:
run(['ciftify_seed_corr',
     '--outputname', os.path.join(seed_corr_dir, '{}_{}_niftitocifti_masked.dscalar.nii'.format(subid, struct)),
     '--mask', cifti_mask,
     func_cifti_smoothed, putamen_vol_seed_mask])

run(['ciftify_meants',
     '--outputcsv', os.path.join(seed_corr_dir, '{}_{}_niftitocifti_masked.csv'.format(subid, struct)),
     '--mask', cifti_mask,
     func_cifti_smoothed, putamen_vol_seed_mask])

subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', hcp_data_dir, 
     os.path.join(seed_corr_dir, '{}_{}_niftitocifti_masked.dscalar.nii'.format(subid, struct)),
     subid, 
     '{}_{}_niftitocifti_masked'.format(subid, struct)])
run_seedcorr_peaktable(os.path.join(seed_corr_dir, '{}_{}_niftitocifti_masked.dscalar.nii'.format(subid, struct)))


Running: ciftify_seed_corr --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_niftitocifti_masked.dscalar.nii --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz
Running: ciftify_meants --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_niftitocifti_masked.csv --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-1

In [30]:
func = func_cifti_smoothed
seed = os.path.join(seed_corr_dir, '{}_{}_cifti.dscalar.nii'.format(subid, struct))
result_type = 'cifti'
result_prefix = '{}_{}_ciftitocifti_unmasked'.format(subid, struct)
run(['wb_command', '-cifti-create-dense-from-template',
     func_cifti_sm0, seed,
     '-volume-all', putamen_vol_seed_mask])

result_map = seed_corr_default_out(func, seed)
run(['ciftify_seed_corr', func, seed]) 


Running: wb_command -cifti-create-dense-from-template /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_cifti.dscalar.nii -volume-all /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz
Running: ciftify_seed_corr /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_cifti.dscalar.nii


0

In [31]:
run(['ciftify_meants', func, seed])
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: ciftify_meants /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_cifti.dscalar.nii
Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_RIGHT-PUTAMEN_cifti.dscalar.nii sub-50005 sub-50005_RIGHT-PUTAMEN_ciftitocifti_unmasked
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_RIGHT-PUTAMEN_cifti.dscalar.nii


In [32]:
result_map = os.path.join(seed_corr_dir, '{}_{}_ciftitocifti_30TRs.dscalar.nii'.format(subid, struct))
result_type = 'cifti'
result_prefix = '{}_{}_ciftitocifti_30TRs'.format(subid, struct)

TR_file = os.path.join(new_outputs, 'rois','TR_file.txt')
with open(TR_file, "w") as text_file:
    text_file.write('''1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30''')

run(['ciftify_seed_corr',
     '--outputname', result_map,
     '--use-TRs', TR_file,
     func_cifti_smoothed, putamen_vol_seed_mask])
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: ciftify_seed_corr --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_ciftitocifti_30TRs.dscalar.nii --use-TRs /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/TR_file.txt /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz
Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_ciftitocifti_30TRs.dscalar.nii sub-50005 sub-50005_RIGHT-PUTAMEN_ciftitocifti_30TRs
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlab

In [33]:
result_map = os.path.join(seed_corr_dir, '{}_{}_ciftitocifti_masked.dscalar.nii'.format(subid, struct))
result_type = 'cifti'
result_prefix = '{}_{}_ciftitocifti_masked'.format(subid, struct)

run(['ciftify_seed_corr',     
     '--outputname', result_map,
     '--mask', cifti_mask,
     func_cifti_smoothed, 
    os.path.join(seed_corr_dir, '{}_{}_cifti.dscalar.nii'.format(subid, struct))])

run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: ciftify_seed_corr --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_ciftitocifti_masked.dscalar.nii --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_cifti.dscalar.nii
Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_ciftitocifti_masked.dscalar.nii sub-50005 sub-50005_RIGHT-PUTAMEN_ciftitocifti_masked
Running: ciftify_peaktable --min-threshold -0

In [114]:
test1 = MeantsTest(func_cifti_smoothed, 
                   os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii'), 
                   subid, 
                   'ciftitocifti', 
                   hcp_data_dir, 
                   outputdir = None, 
                   extra_args = '--weighted', 
                   mask = None)

test2 = MeantsTest(func_cifti_smoothed, 
                   os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii'), 
                   subid, 
                   'ciftitocifti', 
                   hcp_data_dir, 
                   outputdir = seed_corr_dir, 
                   extra_args = '--weighted', 
                   mask = cifti_mask)

Running: ciftify_seed_corr --weighted /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/gaussian_roi.dscalar.nii
Running: ciftify_meants --weighted /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/gaussian_roi.dscalar.nii
Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_gaussian_roi.dscalar.nii sub-50005 rest_test1_Atlas_s12_gaussian_roi_ciftitocifti_umasked
Running: ciftify_peaktable --min-t

In [40]:
result_map = os.path.join(seed_corr_dir, '{}_gaussian_ciftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_gaussian_ciftitocifti_masked'.format(subid)

run(['ciftify_seed_corr', '--weighted', 
     '--outputname', result_map,
     '--mask', cifti_mask,
     func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii')])

run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: ciftify_seed_corr --weighted --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_gaussian_ciftitocifti_masked.dscalar.nii --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/gaussian_roi.dscalar.nii
Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_gaussian_ciftitocifti_masked.dscalar.nii sub-50005 sub-50005_gaussian_ciftitocifti_masked
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluste

In [43]:
L_gaussian_roi = os.path.join(new_outputs, 'rois', 'gaussian_L_roi.shape.gii')

run(['wb_command', '-cifti-separate', 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii'),
     'COLUMN','-metric', 'CORTEX_LEFT', L_gaussian_roi])

Running: wb_command -cifti-separate /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/gaussian_roi.dscalar.nii COLUMN -metric CORTEX_LEFT /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/gaussian_L_roi.shape.gii
Running: ciftify_seed_corr --weighted --hemi L /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/gaussian_L_roi.shape.gii
Running: ciftify_meants --weighted --hemi L /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/gaussian_L_roi.shape.gii
Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-0

In [113]:
test1 = MeantsTest(func_cifti_smoothed, 
                   L_gaussian_roi, 
                   subid, 
                   'giftitocifti', 
                   hcp_data_dir, 
                   outputdir = None, 
                   extra_args = '--weighted --hemi L', 
                   mask = None)

test1 = MeantsTest(func_cifti_smoothed, 
                   L_gaussian_roi, 
                   subid, 
                   'giftitocifti', 
                   hcp_data_dir, 
                   outputdir = seed_corr_dir, 
                   extra_args = '--weighted --hemi L', 
                   mask = cifti_mask)

Running: ciftify_seed_corr --weighted --hemi L /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/gaussian_L_roi.shape.gii
Running: ciftify_meants --weighted --hemi L /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/gaussian_L_roi.shape.gii
Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_gaussian_L_roi.dscalar.nii sub-50005 rest_test1_Atlas_s12_gaussian_L_roi_giftitocifti_umasked
Running: cif

In [112]:
test1 = MeantsTest(func_cifti_smoothed, 
                   os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii'), 
                   subid, 
                   'ciftitocifti', 
                   hcp_data_dir, 
                   outputdir = None, 
                   extra_args = '--weighted', 
                   mask = None)

test1 = MeantsTest(func_cifti_smoothed, 
                   os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii'), 
                   subid, 
                   'ciftitocifti', 
                   hcp_data_dir, 
                   outputdir = seed_corr_dir, 
                   extra_args = '--weighted', 
                   mask = cifti_mask)

Running: ciftify_seed_corr --weighted /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/probmap_roi.dscalar.nii
Running: ciftify_meants --weighted /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/probmap_roi.dscalar.nii
Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_probmap_roi.dscalar.nii sub-50005 rest_test1_Atlas_s12_probmap_roi_ciftitocifti_umasked
Running: ciftify_peaktable --min-thres

In [111]:
R_probmap_roi = os.path.join(new_outputs, 'rois', 'probmap_R_roi.shape.gii')
                                   
run(['wb_command', '-cifti-separate', 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii'),
     'COLUMN','-metric', 'CORTEX_RIGHT', R_probmap_roi])

test1 = MeantsTest(func_cifti_smoothed, 
                   R_probmap_roi, 
                   subid, 
                   'giftitocifti', 
                   hcp_data_dir, 
                   outputdir = None, 
                   extra_args = '--weighted --hemi R', 
                   mask = None)

test1 = MeantsTest(func_cifti_smoothed, 
                   R_probmap_roi, 
                   subid, 
                   'giftitocifti', 
                   hcp_data_dir, 
                   outputdir = seed_corr_dir, 
                   extra_args = '--weighted --hemi R', 
                   mask = cifti_mask)

Running: wb_command -cifti-separate /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/probmap_roi.dscalar.nii COLUMN -metric CORTEX_RIGHT /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/probmap_R_roi.shape.gii
Running: ciftify_seed_corr --weighted --hemi R /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/probmap_R_roi.shape.gii
Running: ciftify_meants --weighted --hemi R /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/probmap_R_roi.shape.gii
Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-2

In [89]:
test1 = MeantsTest(func_cifti_smoothed, 
                   os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii'), 
                   subid, 
                   '7_ciftitocifti', 
                   hcp_data_dir, 
                   outputdir = None, 
                   extra_args = '--roi-label 7', 
                   mask = None)

test1 = MeantsTest(func_cifti_smoothed, 
                   os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii'), 
                   subid, 
                   '7_ciftitocifti', 
                   hcp_data_dir, 
                   outputdir = seed_corr_dir, 
                   extra_args = '--roi-label 7', 
                   mask = cifti_mask)

Running: ciftify_seed_corr --roi-label 7 /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/tvertex.dscalar.nii
Running: ciftify_meants --roi-label 7 /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/tvertex.dscalar.nii
Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_tvertex.dscalar.nii sub-50005 rest_test1_Atlas_s12_tvertex_7_ciftitocifti_umasked
Running: ciftify_peaktable --min-threshold -0.

In [94]:
R_tvertex_roi = os.path.join(new_outputs, 'rois', 'tvertex_R_roi.shape.gii')
run(['wb_command', '-cifti-separate', 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii'),
     'COLUMN','-metric', 'CORTEX_RIGHT', R_tvertex_roi])

test3 = MeantsTest(func_cifti_smoothed, 
                   R_tvertex_roi, 
                   subid, 
                   '7_giftitocifti', 
                   hcp_data_dir, 
                   outputdir = None, 
                   extra_args = '--roi-label 7 --hemi R', 
                   mask = None)

test4 = MeantsTest(func_cifti_smoothed, 
                   R_tvertex_roi, 
                   subid, 
                   '7_giftitocifti', 
                   hcp_data_dir, 
                   outputdir = seed_corr_dir, 
                   extra_args = '--roi-label 7 --hemi R', 
                   mask = cifti_mask)

Running: wb_command -cifti-separate /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/tvertex.dscalar.nii COLUMN -metric CORTEX_RIGHT /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/tvertex_R_roi.shape.gii
Running: ciftify_seed_corr --roi-label 7 --hemi R /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/tvertex_R_roi.shape.gii
Running: ciftify_meants --roi-label 7 --hemi R /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/tvertex_R_roi.shape.gii
Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09

In [95]:
run(['cifti_vis_map', 'index', '--hcp-data-dir', hcp_data_dir])

Running: cifti_vis_map index --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp


0

# ciftify_meants (atlas examples)

In [98]:
subject_aparc = os.path.join(hcp_data_dir, subid,
                            'MNINonLinear', 'fsaverage_LR32k',
                            '{}.aparc.32k_fs_LR.dlabel.nii'.format(subid))
subject_thickness = os.path.join(hcp_data_dir, subid,
                            'MNINonLinear', 'fsaverage_LR32k',
                            '{}.thickness.32k_fs_LR.dscalar.nii'.format(subid))
run(['ciftify_meants', func_cifti_sm0, subject_aparc])


Running: ciftify_meants /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.aparc.32k_fs_LR.dlabel.nii


0

In [99]:
run(['ciftify_meants',  
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_aparc_thickness_ciftitocifti_unmasked_meants.csv'.format(subid)),
     '--outputlabels', os.path.join(seed_corr_dir, 
                                    '{}_aparc_ciftitocifti_unmasked_labels.csv'.format(subid)),
         subject_thickness, subject_aparc])

Running: ciftify_meants --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_aparc_thickness_ciftitocifti_unmasked_meants.csv --outputlabels /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_aparc_ciftitocifti_unmasked_labels.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.thickness.32k_fs_LR.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.aparc.32k_fs_LR.dlabel.nii


0

In [100]:
run(['ciftify_meants', func_cifti_sm0, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

Running: ciftify_meants /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/tvertex.dscalar.nii


0

In [101]:
run(['ciftify_meants', 
     '--mask', cifti_mask,
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_tvertex_func_ciftitocifti_unmasked_meants.csv'.format(subid)),
     func_cifti_sm0, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

Running: ciftify_meants --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_tvertex_func_ciftitocifti_unmasked_meants.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/rois/tvertex.dscalar.nii


0

In [102]:
## project wmparc to subject
wmparc_dscalar_d0 = os.path.join(seed_corr_dir, '{}_wmparc_MNI_d0.dscalar.nii'.format(subid))
run(['ciftify_vol_result', '--hcp-data-dir', hcp_data_dir,
    '--integer-labels', subid, atlas_vol, wmparc_dscalar_d0])

Running: ciftify_vol_result --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp --integer-labels sub-50005 /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/wmparc.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_wmparc_MNI_d0.dscalar.nii


0

In [103]:
## project wmparc to subject
wmparc_dscalar = os.path.join(seed_corr_dir, '{}_wmparc_MNI_d10.dscalar.nii'.format(subid))
run(['ciftify_vol_result', 
     '--hcp-data-dir', hcp_data_dir,
     '--dilate', '10',
    '--integer-labels', subid, atlas_vol, wmparc_dscalar])

Running: ciftify_vol_result --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp --dilate 10 --integer-labels sub-50005 /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/wmparc.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_wmparc_MNI_d10.dscalar.nii


0

In [104]:
run(['ciftify_meants', 
     '--mask', os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'brainmask_fs.nii.gz'),
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_niftitonifti_masked_meants.csv'.format(subid)),
     '--outputlabels', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_niftitonifti_masked_labels.csv'.format(subid)),
     func_vol, atlas_vol])

Running: ciftify_meants --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/brainmask_fs.nii.gz --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_wmparc_func_niftitonifti_masked_meants.csv --outputlabels /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_wmparc_func_niftitonifti_masked_labels.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/wmparc.nii.gz


0

In [105]:
run(['ciftify_meants', 
     '--mask', cifti_mask,
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_ciftitocifti_masked_meants.csv'.format(subid)),
     '--outputlabels', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_ciftitocifti_masked_labels.csv'.format(subid)),
     func_cifti_sm0, wmparc_dscalar])

Running: ciftify_meants --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_wmparc_func_ciftitocifti_masked_meants.csv --outputlabels /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_wmparc_func_ciftitocifti_masked_labels.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_wmparc_MNI_d10.dscalar.nii


0

In [106]:
wmparc_dlabel = os.path.join(seed_corr_dir, '{}_wmparc_MNI.dlabel.nii'.format(subid))
run(['wb_command', '-cifti-label-import', '-logging', 'SEVERE',
     wmparc_dscalar, 
     os.path.join(ciftify.config.find_ciftify_global(), 'hcp_config', 'FreeSurferAllLut.txt'),
    wmparc_dlabel])
run(['ciftify_meants', 
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_ciftiltocifti_dlabel_meants.csv'.format(subid)),
     '--outputlabels', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_ciftiltocifti_dlabel_labels.csv'.format(subid)),
     func_cifti_sm0, wmparc_dlabel])

Running: wb_command -cifti-label-import -logging SEVERE /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_wmparc_MNI_d10.dscalar.nii /projects/edickie/code/ciftify/ciftify/data/hcp_config/FreeSurferAllLut.txt /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_wmparc_MNI.dlabel.nii
Running: ciftify_meants --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_wmparc_func_ciftiltocifti_dlabel_meants.csv --outputlabels /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005_wmparc_func_ciftiltocifti_dlabel_labels.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-25_py36_sc2017-10-13/ciftify_seed_corr/sub-50005

0

In [107]:
import pandas as pd

csv_df = pd.read_csv(os.path.join(fixtures_dir, 'expected_csvs.csv'))

csv_df['num_rows'] = ''
csv_df['num_cols'] = ''
csv_df['exists'] = ''
csv_df['matches'] = ''

In [108]:
for row in csv_df.index:
    expected_csv = os.path.join(new_outputs, csv_df.loc[row, 'expected_output'])
    if os.path.isfile(expected_csv):
        compare_csv = os.path.join(fixtures_dir, csv_df.loc[row, 'compare_to'])
        header_col = 0
        if expected_csv.endswith('_meants.csv'): header_col = None
        if expected_csv.endswith('_labels.csv'): header_col = None
        if expected_csv.endswith('_CSF.csv'): header_col = None
        if expected_csv.endswith('_WM.csv'): header_col = None
        if expected_csv.endswith('_GM.csv'): header_col = None
        testdf = pd.read_csv(expected_csv, header = header_col)
        csv_df.loc[row, 'exists'] = True
        csv_df.loc[row, 'num_rows'] = testdf.shape[0]
        csv_df.loc[row, 'num_cols'] = testdf.shape[1]
        comparedf = pd.read_csv(compare_csv, header = header_col)
        try:
            csv_df.loc[row, 'matches'] = (comparedf == testdf).all().all()
        except:
            csv_df.loc[row, 'matches'] = False
    else:
        csv_df.loc[row, 'exists'] = False

In [ ]:
csv_df.to_csv(os.path.join(new_outputs, 'csv_compare_results.csv'))

In [ ]:
if csv_df.exists.all(): 
    logger.info('All expected csv were generated')
else:
    logger.error('Some expected csv outputs were not generated')

In [ ]:
if csv_df.matches.all():
    logger.info('All expected csv outputs match expected')
else:
    logger.info('Some csv outputs do not match..see csv_compare_results.csv of more details')

In [ ]:
logger.info(ciftify.utils.section_header('Done'))

In [ ]:
csv_df.loc[csv_df.matches == False]

In [110]:
csv_df

,compare_to,expected_output,num_rows,num_cols,exists,matches
0,./ciftify_seed_corr/sub-50005_aparc_ciftitocif...,./ciftify_seed_corr/sub-50005_aparc_ciftitocif...,71,6,True,True
1,./ciftify_seed_corr/sub-50005_aparc_thickness_...,./ciftify_seed_corr/sub-50005_aparc_thickness_...,68,1,True,True
2,./hcp/sub-50005/MNINonLinear/Results/rest_test...,./ciftify_seed_corr/sub-50005_gaussian_ciftito...,26,14,True,True
3,./hcp/sub-50005/MNINonLinear/Results/rest_test...,./ciftify_seed_corr/sub-50005_gaussian_ciftito...,5,1,True,True
4,./hcp/sub-50005/MNINonLinear/Results/rest_test...,./ciftify_seed_corr/sub-50005_gaussian_giftito...,26,14,True,True
5,./hcp/sub-50005/MNINonLinear/Results/rest_test...,./ciftify_seed_corr/sub-50005_gaussian_giftito...,5,1,True,True
6,./hcp/sub-50005/MNINonLinear/Results/rest_test...,./ciftify_seed_corr/sub-50005_probmap_ciftitoc...,12,14,True,True
7,./hcp/sub-50005/MNINonLinear/Results/rest_test...,./ciftify_seed_corr/sub-50005_probmap_ciftitoc...,0,1,True,True
8,./hcp/sub-50005/MNINonLinear/Results/rest_test...,./ciftify_seed_corr/sub-50005_probmap_giftitoc...,12,14,True,True
9,./hcp/sub-50005/MNINonLinear/Results/rest_test...,./ciftify_seed_corr/sub-50005_probmap_giftitoc...,0,1,True,True
